In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
from google.colab import drive, files # google colab specific
import requests
import pandas as pd
import os
import warnings
import sys
import matplotlib.pyplot as plt
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tensorflow import keras
from keras import layers
import math
from sklearn.metrics import mean_squared_error
from datetime import datetime
import datetime as dt
from pylab import rcParams



In [ ]:
# Read Data from CSV into Dataframe
df = pd.read_csv('/content/drive/MyDrive/JPM/jphomevalue/data/RealEstateFundamentalData2005-.csv')
df.head()

In [ ]:
# Each column represents a house and it's historical prices
# Isolate a Date|House pair and store it into another df  

In [ ]:
# Reset Index, Perform Pre-Processing on the Data

In [ ]:
# Upload the prediction model here and predict the data


In [ ]:
# Append the data to another dataframe  

In [ ]:
# After the new dataframe is completely filled with prediction data transform to CSV  

In [ ]:
# Download final CSV file and it is ready to populate to DB